In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dfhelper as dfh
import text_preproc as tp

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

In [ ]:
pre_df = pd.read_pickle('dataset/train_df_v2.pkl')

In [ ]:
pre_df.shape

In [ ]:
dfh.printColNames(pre_df)

In [ ]:
marr = pre_df.columns.values

title = marr[0]
bull_pt = marr[1]
desc = marr[2]
prod_type_id= marr[3]
prod_len = marr[4]


In [ ]:
cpy_df = pre_df.copy()

In [ ]:
# pre_df = pre_df.reset_index()

In [ ]:
cpy_df = pd.get_dummies(pre_df, columns=[prod_type_id])

In [ ]:
cpy_df
cpy_df.shape

In [ ]:
first_thousand = pre_df[:1000]
first_thousand

In [ ]:
import keyboard

In [ ]:
# for i in range(1000):
#     print(pre_df.iloc[i]["TITLE"])
#     keyboard.wait('/')

In [ ]:
mlist = []

In [ ]:
# mdf = pre_df.drop(columns=['BULLET_POINTS', 'DESCRIPTION', 'PRODUCT_LENGTH'])
# mdf.head()

In [ ]:
mlist = []
# back prop
# we are obviously smart
# let's see how we predict and the try to extract
# print all with prod_len between 0 and 100
def printCountInGivenRange(pre_df, inr, outr):
    count = 0
    # for i in range(pre_df.shape[0]):
    for i in range(1000):
        if pre_df.iloc[i][prod_len] <= outr and pre_df.iloc[i][prod_len] >= inr:
            count += 1
            print(i, pre_df.iloc[i][prod_len], pre_df.iloc[i][title])
            mlist.append(i)
            num = ""
            for j in pre_df.columns.values:
                if ("id" in j.lower()):
                    num += str(pre_df.iloc[i][j]) + " "
            print(num)
                    



In [ ]:
printCountInGivenRange(first_thousand, 0, 100)

In [ ]:
printCountInGivenRange(first_thousand, 100, 200)

In [ ]:
pre_df.max()

In [ ]:
# test_mrow = pre_df[prod_len]
for i,v in first_thousand.iterrows():
    if (v[prod_len] == 1885801400.0):
        print(v[title])
        print(v)

In [ ]:
# simply put through lazy predict bus

In [ ]:
# # use cpy_df. First do splits and then use lazy predict
# %pip install lazypredict

In [ ]:
# from lazypredict.Supervised import LazyClassifier
# from sklearn.datasets import load_breast_cancer
# from sklearn.model_selection import train_test_split

# data = load_breast_cancer()
# X = data.data
# y= data.target

# X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)

# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(X_train, X_test, y_train, y_test)

# print(models)

In [ ]:
# %pip install gensim

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import gensim
from lazypredict.Supervised import LazyClassifier

In [ ]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim

# Download the required NLTK resources
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove numbers and punctuation
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize text into words
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize words
    lem = WordNetLemmatizer()
    words = [lem.lemmatize(word, "v") for word in words]

    # Join words back into a sentence
    preprocessed_text = " ".join(words)

    return preprocessed_text

# Example usage:
text = "This is an example sentence. It will be used to demonstrate text preprocessing for Word2Vec."
preprocessed_text = preprocess_text(text)
print(preprocessed_text)

In [ ]:
# Load data
# data = pd.read_csv('your_data.csv')
data = cpy_df.copy()


In [ ]:

# Split data
X_train, X_test, y_train, y_test = train_test_split(data.drop([prod_len], axis=1), data[prod_len], test_size=0.2, random_state=42)


In [ ]:

# Load and prepare word embeddings
# w2v_model = Word2Vec.load('your_word_embeddings.model')
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('models/GoogleNews-vectors-negative300.bin.gz', binary=True)


In [ ]:
# maybe you need to pre process the data
# maybe you need to use ann and tensorflow instead of sklearn
m_word = X_train[title].values[0].split(" ")[0]
print(m_word)

In [ ]:
w2v_model
word = 'king'
vector = w2v_model[m_word]
print(vector)

In [ ]:
# %pip install transformers

In [ ]:
# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)


In [93]:

# Tokenize input text and convert to TensorFlow tensors
# text = "This is an example sentence."
text = "This is an example sentence. Tokenize input text and convert to TensorFlow tensors "
# inputs = tokenizer(text, return_tensors='tf')
inputs = tokenizer(text, return_tensors='tf', max_length=512, truncation=True)
# inputs = tokenizer(text)

# Pass input through the model to get the output vector
# outputs = model(inputs)[0]
outputs = model(inputs)

# Print the output vector
print(outputs[0])

tf.Tensor(
[[[-0.39459726 -0.785355   -0.3071021  ... -0.855016   -0.2719474
    0.5896925 ]
  [-0.74061793 -0.60996133 -0.5062999  ... -0.31280124  0.7690935
    0.07103826]
  [-0.36278898 -0.6912678   0.2634068  ... -0.07483993  0.523368
    0.58124304]
  ...
  [ 0.2371747   0.26732197  0.37201953 ... -0.638582   -1.0357348
    0.66088116]
  [-0.30154967 -0.36846936 -0.00316174 ... -0.6946786  -0.73371553
   -0.04277612]
  [ 0.6092344  -0.1332865  -0.40263057 ...  0.23701602 -0.5503922
    0.02643796]]], shape=(1, 19, 768), dtype=float32)


In [100]:
# Example list of texts
# texts = ['This is the first text.', 'This is the second text.', 'This is the third text.']
texts = ["HACASO - The Princess Sleeps Here- Pink Wall Decal Girl's Bedroom Decor", 'PINKCITY CREATION Handmade Showpieces Figurine Fluorite Stone Religious Idol Ganesha 5 cm.', 'Lovely Furniture Floating Mounted Cube Shape Wall Shelves Set of 4 (Black)']

# Tokenize the texts
# tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=512)
tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=512)
outputs = model(inputs)
# Print the tokenized texts
print(outputs)

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 19, 768), dtype=float32, numpy=
array([[[-0.39459726, -0.785355  , -0.3071021 , ..., -0.855016  ,
         -0.2719474 ,  0.5896925 ],
        [-0.74061793, -0.60996133, -0.5062999 , ..., -0.31280124,
          0.7690935 ,  0.07103826],
        [-0.36278898, -0.6912678 ,  0.2634068 , ..., -0.07483993,
          0.523368  ,  0.58124304],
        ...,
        [ 0.2371747 ,  0.26732197,  0.37201953, ..., -0.638582  ,
         -1.0357348 ,  0.66088116],
        [-0.30154967, -0.36846936, -0.00316174, ..., -0.6946786 ,
         -0.73371553, -0.04277612],
        [ 0.6092344 , -0.1332865 , -0.40263057, ...,  0.23701602,
         -0.5503922 ,  0.02643796]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-0.73170996, -0.4720331 , -0.91274244,  0.5767257 ,  0.8155095 ,
        -0.41480705,  0.5162307 ,  0.27219692, -0.8038374 , -0.9999446 ,
        -0.48134282,  0.762

In [ ]:
# text = X_train[title].values[0]
text = X_train[title].values[0:3].tolist()
print(type(text))
print(text)


In [ ]:
# inputs = tokenizer(text, return_tensors='tf')
inputs = tokenizer(text, return_tensors='tf', padding=True, truncation=True, max_length=512)
# inputs = tokenizer(text)

# Pass input through the model to get the output vector
outputs = model(inputs)

# Print the output vector
print(outputs[0])

In [ ]:
# Convert output tensor to numpy array
output_array = outputs.numpy()

# Flatten and reshape the output array to match sklearn input format
output_array = output_array.flatten().reshape(1, -1)

# Print the output array
print(output_array)
output_array.shape

In [ ]:

msent = preprocess_text(msent)
print(msent)
msent = msent.split(" ")
print(msent)
# msent = [w2v_model[w] for w in msent]
msent2 = []
for i in msent:
    try:
        msent2.append(w2v_model[i])
    except Exception as e:
        print(e)
        print(i)

print(msent2)

In [ ]:

msent = preprocess_text(msent)
print(msent)
msent = msent.split(" ")
print(msent)
# msent = [w2v_model[w] for w in msent]
msent2 = []
for i in msent:
    try:
        msent2.append(w2v_model[i])
    except Exception as e:
        print(e)
        print(i)

print(msent2)

In [ ]:

X_train_embeddings = [w2v_model.wv[word] for word in X_train[title]]
X_test_embeddings = [w2v_model.wv[word] for word in X_test[title]]


In [ ]:
# %pip install nltk

In [ ]:

X_train_embeddings2 = [w2v_model.wv[word] for word in X_train[desc]]
X_test_embeddings2 = [w2v_model.wv[word] for word in X_test[desc]]

# Merge embeddings with other features
X_train_final = pd.concat([X_train.drop([title], axis=1), pd.DataFrame(X_train_embeddings)], axis=1)
X_test_final = pd.concat([X_test.drop([title], axis=1), pd.DataFrame(X_test_embeddings)], axis=1)

# Merge embeddings with other features
X_train_final = pd.concat([X_train.drop([desc], axis=1), pd.DataFrame(X_train_embeddings2)], axis=1)
X_test_final = pd.concat([X_test.drop([desc], axis=1), pd.DataFrame(X_test_embeddings2)], axis=1)

# Use Lazy Predict
clf = LazyClassifier(verbose=2,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train_final, X_test_final, y_train, y_test)

print(models)


In [ ]:

X_train_embeddings2 = [w2v_model.wv[word] for word in X_train[desc]]
X_test_embeddings2 = [w2v_model.wv[word] for word in X_test[desc]]

# Merge embeddings with other features
X_train_final = pd.concat([X_train.drop([title], axis=1), pd.DataFrame(X_train_embeddings)], axis=1)
X_test_final = pd.concat([X_test.drop([title], axis=1), pd.DataFrame(X_test_embeddings)], axis=1)

# Merge embeddings with other features
X_train_final = pd.concat([X_train.drop([desc], axis=1), pd.DataFrame(X_train_embeddings2)], axis=1)
X_test_final = pd.concat([X_test.drop([desc], axis=1), pd.DataFrame(X_test_embeddings2)], axis=1)

# Use Lazy Predict
clf = LazyClassifier(verbose=2,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train_final, X_test_final, y_train, y_test)

print(models)


In [121]:
def mtokenize2(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='tf', max_length=512)
    outputs = model(inputs)
  
    return outputs[0]

In [114]:
def mtokenize(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='tf', max_length=512)
    outputs = model(inputs)

    output_array = outputs[0].numpy()

    # Flatten and reshape the output array to match sklearn input format
    output_array = output_array.flatten().reshape(1, -1)
    
    # return outputs[0]
    return output_array

In [118]:
texts = ["HACASO - The Princess Sleeps Here- Pink Wall Decal Girl's Bedroom Decor", 'PINKCITY CREATION Handmade Showpieces Figurine Fluorite Stone Religious Idol Ganesha 5 cm.', 'Lovely Furniture Floating Mounted Cube Shape Wall Shelves Set of 4 (Black)']

In [122]:
for i in texts:
    print(mtokenize(i))
    print(mtokenize(i).shape)

    print(mtokenize2(i))
    print(mtokenize2(i).shape)

[[-0.43380743 -0.594643    0.07364444 ...  0.36776006 -0.38914543
  -0.22698034]]
(1, 15360)
tf.Tensor(
[[[-0.43380743 -0.594643    0.07364444 ... -0.05213424  0.12490535
    0.42825395]
  [-0.10545568 -1.6703563   0.23767257 ... -0.11654496  0.26866344
    0.2958064 ]
  [-1.2422262  -0.7605037   0.44002134 ... -0.5697189   0.00459877
   -0.8156161 ]
  ...
  [ 0.84935945  0.1251586  -0.13403404 ... -0.15713198 -0.01781187
   -0.42972186]
  [ 0.5802119   0.4472033   0.36661974 ... -0.422646   -0.44749838
    0.26063016]
  [ 0.6544737  -0.28135595 -0.01946732 ...  0.36776006 -0.38914543
   -0.22698034]]], shape=(1, 20, 768), dtype=float32)
(1, 20, 768)
[[-0.6215099  -0.28986043 -0.10749999 ...  0.07650582 -0.29404986
  -0.18917343]]
(1, 19200)
tf.Tensor(
[[[-0.6215099  -0.28986043 -0.10749999 ... -0.18282071  0.21790671
    0.22730517]
  [ 0.12478697 -0.65829     0.46100584 ... -0.15136173  0.38925788
   -0.48844454]
  [ 0.42804483 -0.8235196   0.26472005 ... -0.19160193 -0.11209712
   -

In [116]:
mtokenize("hansco this")

array([[-0.35298762,  0.02311424,  0.06549785, ..., -0.0628305 ,
        -0.7847901 , -0.30727923]], dtype=float32)

In [117]:
mtokenize("hansco this")[0].shape

(3840,)

In [123]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
